In [1]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
init_printing(use_latex='mathjax')

# Asymptotic Models

One thing a reader may have noticed if they have been running the code throughout is that it can take Sympy some time to solve our equations analytically, which is worrying considering that the equations given to Sympy to solve in these examples are quite simple, and could be done with relative ease by the mathematician. And this is true. But in the physical sciences, asmyptotics can be used to rigorously simplify models, and having a tool to automate those calculations allows the end user to spend time validating the model. This chapter will go over some examples of how Sympy can be used to derive these models

## Prandtl Boundary Layer Theory

This work on weakly viscous fluids near a boundary from 1905 is the original singular perturbation problem that can be treated by MMAE.

We begin with the non-dimensional, incompressible, two-dimensional Navier-Stokes equations $$u_t + uu_x + vu_y + p_x = \epsilon(u_{xx}+u_{yy}), \\ v_t + uv_x + vv_y + p_y = \epsilon(v_{xx}+v_{yy}), \\ u_x+v_y = 0,$$
where $\epsilon$ is the inverse Reynold's number, $u,v$ are the velocities in the $x,y$ directions, and $p$ is the pressure. The $\epsilon = 0$ case are known as the Euler equations. Since both are first order in time, they can both take the same initial conditions $u(x,y,0) = u_0(x,y), v(x,y,0) = v_0(x,y),$ and no initial condition is needed for the pressure. However, since the Euler and NS equations have different orders of spatial derivatives, different boundary conditions are required. The Euler equations require the no-flow condition $\langle u,v\rangle\cdot\mathbf{n}=0$ for normal vector $\mathbf{n},$ which negates velocities flowing out from a region. Tthe Navier Stokes equations require the no-slip boundary conditions $\langle u,v\rangle = 0,$ negating both the tangential and normal velocities on the boundary.

In particular, we will consider a flat boundary at $y=0$ (the boundary layer is thin enough that the equations would not change for a smooth curved boundary).

The leading order outer solution becomes from the Euler equations obtained when $\epsilon = 0,$ which is located in $y>0$. We will denote the outer solution using lowercased variables. 

For the outer solution, we will use uppercased variables, which are unscaled except $Y = \frac y\delta, v = \eta V.$ The following code will automate the generation of the scaled equations from our definitions. 

Among the code are several filters streamline the process of substitution, one of which (scaling) is set to a null dictionary, with the actual scaling commented out. Looking at the expression, one arrives at the dominant balance $\eta = \delta = \sqrt{\epsilon}$. By uncommenting the dictionary scaling, and the collect call on the inner line, one will arrive at the leading order outer and inner equations.

In [22]:
epsilon = symbols('epsilon')
x, y, t = symbols('x y t')
X, Y, T = symbols('X Y T')
u, p = [Function(c)(x,y,t) + Function(c.upper())(X,Y,T) for c in ['u','p']]
ddx = lambda f: f.diff(x) + f.diff(X)
ddt = lambda f: f.diff(t) + f.diff(T)

scales = symbols('eta delta')
v = Function('v')(x,y,t) + scales[0]*Function('V')(X,Y,T)
ddy = lambda f: f.diff(y) + f.diff(Y)/scales[1]

eqns = []
eqns.append(ddt(u) + u*ddx(u) + v*ddy(u) + ddx(p) - epsilon*(ddx(ddx(u)) + ddy(ddy(u))))
eqns.append(ddt(v) + u*ddx(v) + v*ddy(v) + ddy(p) - epsilon*(ddx(ddx(v)) + ddy(ddy(v))))
eqns.append(ddx(u) + ddy(v))

#generate filters for substitution
no_upper = {Function(c)(X,Y,T):0 for c in ['U','V','P']}
no_lower = {Function(c)(x,y,t):0 for c in ['u','v','p']}
eps12 = epsilon**Rational(1,2)
scaling = {}#{scales[0]:eps12,scales[1]:eps12}

for eqn in eqns:
    eqn = eqn.expand()
    outer = eqn.subs(no_upper).simplify().collect(epsilon,evaluate=False)
    display(outer)
    inner = eqn.subs(no_lower).subs(scaling).simplify()#.collect(epsilon,evaluate=False)
    display(inner)




⎧                                                                              ↪
⎪              ∂                           ∂                ∂                ∂ ↪
⎨1: u(x, y, t)⋅──(u(x, y, t)) + v(x, y, t)⋅──(u(x, y, t)) + ──(p(x, y, t)) + ─ ↪
⎪              ∂x                          ∂y               ∂x               ∂ ↪
⎩                                                                              ↪

↪                      2                 2             ⎫
↪                     ∂                 ∂              ⎪
↪ ─(u(x, y, t)), ε: - ───(u(x, y, t)) - ───(u(x, y, t))⎬
↪ t                     2                 2            ⎪
↪                     ∂x                ∂y             ⎭

⎧                                                                              ↪
⎪              ∂                           ∂                ∂                ∂ ↪
⎨1: U(X, Y, T)⋅──(U(X, Y, T)) + V(X, Y, T)⋅──(U(X, Y, T)) + ──(P(X, Y, T)) + ─ ↪
⎪              ∂X                          ∂Y               ∂X               ∂ ↪
⎩                                                                              ↪

↪                  2                    2             ⎫
↪                 ∂                    ∂              ⎪
↪ ─(U(X, Y, T)) - ───(U(X, Y, T)), ε: -───(U(X, Y, T))⎬
↪ T                 2                    2            ⎪
↪                 ∂Y                   ∂X             ⎭

⎧                                                                              ↪
⎪              ∂                           ∂                ∂                ∂ ↪
⎨1: u(x, y, t)⋅──(v(x, y, t)) + v(x, y, t)⋅──(v(x, y, t)) + ──(p(x, y, t)) + ─ ↪
⎪              ∂x                          ∂y               ∂y               ∂ ↪
⎩                                                                              ↪

↪                      2                 2             ⎫
↪                     ∂                 ∂              ⎪
↪ ─(v(x, y, t)), ε: - ───(v(x, y, t)) - ───(v(x, y, t))⎬
↪ t                     2                 2            ⎪
↪                     ∂x                ∂y             ⎭

⎧                                                                              ↪
⎪1   ∂                              ∂                           ∂              ↪
⎨──: ──(P(X, Y, T)), √ε: U(X, Y, T)⋅──(V(X, Y, T)) + V(X, Y, T)⋅──(V(X, Y, T)) ↪
⎪√ε  ∂Y                             ∂X                          ∂Y             ↪
⎩                                                                              ↪

↪                      2                       2             ⎫
↪    ∂                ∂                 3/2   ∂              ⎪
↪  + ──(V(X, Y, T)) - ───(V(X, Y, T)), ε   : -───(V(X, Y, T))⎬
↪    ∂T                 2                       2            ⎪
↪                     ∂Y                      ∂X             ⎭

⎧   ∂                ∂             ⎫
⎨1: ──(u(x, y, t)) + ──(v(x, y, t))⎬
⎩   ∂x               ∂y            ⎭

⎧   ∂                ∂             ⎫
⎨1: ──(U(X, Y, T)) + ──(V(X, Y, T))⎬
⎩   ∂X               ∂Y            ⎭

From examining and matching the leading order equations, we are able to build out the rest of the analysis. Since the inner boundary layer solution as $Y\to\infty$ must match the outer Euler solution as $y\to0$ we get $U(X,Y,T)\to U^*(X,T):=u(X,0,T), P(X,T) = P^*(X,T):= P(X,0,T)$ (Note that $P_Y=0$ makes the latter equality). Since this operates at $y=0, v=0,$ thus we get the compatibility relation $$U^*_T + U^*U^*_X +P^*_X=0.$$ From incompressibility, we get both that $$V(X,Y,T)\sim U_X^*(X,T)Y\text{ as }Y\to\infty$$ $$v(x,y,t)\sim v_y(x,0,t)y = u_x(x,0,t)y \text{ as }y\to0.$$ As such, we do not require matching conditions on $V.$ We also match the intial data to the inner layer to get $U(X,Y,0) = u_0(X,Y).$ For steady flow with $U>0$, this reduces our equations to $$UU_X - U^*U^*_X + VU_Y = U_{YY}$$ $$U_X+V_Y = 0$$ $$U(X,Y)\to U^*(X) \text{ as } Y\to\infty$$ $$U(X,Y)\sim U_0(Y)$$

